In [1]:
import numpy as np
import pandas as pd
import logging
import os
import sys
import re
import shutil
import copy
from datetime import datetime, timedelta
from time import strptime
from sqlalchemy import create_engine
from dateutil.relativedelta import relativedelta
import pymssql
from openpyxl import load_workbook
import calendar


pd.options.display.max_columns = 150

# Read

In [2]:
data = pd.read_csv('./train.csv', encoding = 'big5')

In [49]:
data

,日期,測站,測項,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,12,15,17,20,22,22,22,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,0.59,0.52,0.41,0.4,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,0.2,0.18,0.12,0.11,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,4.2,2.9,3.4,3,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,14,11,14,12,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,18,14,17,15,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,22,21,29,44,58,50,57,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,2,11,38,56,64,56,57,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,20,19,30,41,44,33,37,36,45,42,49,45,44,41,30,24,13


In [6]:
data[data=='NR'] = 0

In [ ]:
for i in

In [20]:
raw_data = data.iloc[:,3:].to_numpy()
raw_data

array([['14', '14', '14', ..., '15', '15', '15'],
       ['1.8', '1.8', '1.8', ..., '1.8', '1.8', '1.8'],
       ['0.51', '0.41', '0.39', ..., '0.35', '0.36', '0.32'],
       ...,
       ['36', '55', '72', ..., '118', '100', '105'],
       ['1.9', '2.4', '1.9', ..., '1.5', '2', '2'],
       ['0.7', '0.8', '1.8', ..., '1.6', '1.8', '2']], dtype=object)

In [35]:
month_data = {}
for month in range(12):
    sample = np.zeros([18, 20*24])
    for day in range(20):
        sample[:,day*24:(day+1)*24] = raw_data[(month*20 + day)*18:(month*20+day+1)*18,:]
    month_data[month] = sample

In [50]:
pd.DataFrame(month_data[1])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479
0,15.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,16.00,20.00,22.00,23.00,24.00,24.00,24.00,23.00,23.00,22.00,19.00,18.00,19.00,18.00,17.00,19.00,18.00,17.00,18.00,18.00,18.00,18.00,17.00,18.00,21.00,24.00,24.00,25.00,26.00,25.00,25.00,25.00,24.00,23.00,22.00,22.00,21.00,20.00,20.00,20.00,20.00,19.00,19.00,19.00,19.00,18.00,18.00,19.00,21.00,23.00,25.00,26.00,26.00,26.00,26.00,25.00,25.00,23.00,22.00,22.00,21.00,20.00,19.00,19.00,18.00,18.00,17.00,...,20.00,20.00,21.00,19.00,19.00,19.00,19.00,19.00,18.00,18.00,18.00,20.00,22.00,24.00,25.00,23.00,21.00,20.00,19.00,18.00,17.00,17.00,15.00,15.00,14.00,13.00,14.00,13.00,13.00,13.00,12.00,12.00,12.00,12.00,11.00,12.00,11.00,11.00,10.00,10.00,10.00,11.00,11.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,9.80,9.30,8.70,8.80,11.00,12.00,13.00,14.00,15.00,15.00,15.00,15.00,14.00,12.00,10.00,9.40,9.10,8.40,8.00,7.60
1,1.80,1.80,1.70,1.80,1.70,1.70,1.70,1.70,1.80,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.70,1.80,1.80,1.70,1.70,1.70,1.80,1.80,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.80,1.80,1.70,1.80,1.80,1.80,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.80,1.80,1.70,1.70,1.70,...,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.70,1.70,1.70,1.70,1.80,1.80,1.80,1.70,1.80,1.70,1.70,1.70,1.80,1.80,1.80,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.70,1.80,1.80,1.80,1.70,1.70,1.70,1.70
2,0.27,0.26,0.25,0.26,0.24,0.23,0.26,0.45,0.76,0.56,0.33,0.30,0.27,0.26,0.25,0.24,0.24,0.33,0.40,0.43,0.33,0.51,0.37,0.33,0.30,0.25,0.31,0.28,0.27,0.25,0.28,0.36,0.43,0.24,0.33,0.31,0.27,0.26,0.28,0.30,0.37,0.38,0.42,0.50,0.45,0.38,0.55,0.46,0.54,0.48,0.39,0.36,0.36,0.35,0.32,0.45,0.44,0.40,0.41,0.36,0.29,0.26,0.26,0.25,0.24,0.23,0.32,0.35,0.32,0.38,0.37,0.32,0.29,0.27,0.31,...,0.77,0.57,0.48,0.41,0.36,0.34,0.32,0.29,0.30,0.35,0.57,0.98,0.90,0.74,1.73,0.58,0.41,0.41,0.45,0.54,0.48,0.47,0.45,0.40,0.39,0.38,0.38,0.35,0.33,0.33,0.32,0.30,0.30,0.32,0.35,0.35,0.36,0.37,0.36,0.37,0.38,0.35,0.37,0.39,0.46,0.41,0.38,0.37,0.36,0.34,0.33,0.32,0.31,0.30,0.29,0.29,0.28,0.33,0.51,0.33,0.29,0.32,0.32,0.32,0.31,0.31,0.33,0.35,0.42,0.43,0.44,0.35,0.36,0.35,0.32
3,0.05,0.05,0.04,0.05,0.05,0.05,0.05,0.13,0.22,0.16,0.06,0.05,0.04,0.04,0.04,0.03,0.04,0.08,0.12,0.14,0.09,0.20,0.10,0.08,0.08,0.06,0.11,0.10,0.10,0.09,0.10,0.11,0.13,0.04,0.08,0.07,0.06,0.06,0.07,0.07,0.08,0.09,0.12,0.18,0.15,0.12,0.21,0.15,0.21,0.16,0.11,0.11,0.07,0.06,0.07,0.10,0.09,0.07,0.08,0.06,0.05,0.05,0.04,0.04,0.05,0.05,0.08,0.09,0.08,0.12,0.11,0.09,0.06,0.05,0.04,...,0.27,0.18,0.17,0.17,0.13,0.09,0.19,0.09,0.16,0.11,0.19,0.56,0.53,0.38,0.27,0.17,0.11,0.14,0.23,0.27,0.16,0.17,0.21,0.13,0.09,0.09,0.12,0.11,0.08,0.10,0.07,0.07,0.08,0.10,0.09,0.11,0.18,0.17,0.17,0.14,0.20,0.23,0.22,0.16,0.18,0.16,0.14,0.15,0.12,0.12,0.14,0.07,0.11,0.07,0.07,0.10,0.12,0.09,0.13,0.09,0.08,0.08,0.09,0.09,0.11,0.10,0.12,0.15,0.17,0.13,0.20,0.08,0.15,0.07,0.06
4,0.90,0.90,1.00,1.60,1.30,1.20,1.40,2.20,4.20,4.30,2.80,2.10,2.10,2.00,1.70,1.70,1.70,1.60,1.60,1.60,1.80,1.80,1.80,1.80,1.90,1.60,1.70,0.70,0.70,0.50,0.80,1.30,3.00,0.90,1.50,1.90,1.70,1.30,1.60,1.50,2.20,0.80,0.80,1.50,1.40,0.90,1.30,0.70,1.20,1.10,0.90,1.90,1.80,2.00,2.00,4.70,6.10,5.60,4.70,4.20,3.20,2.70,1.9

In [75]:
x = np.empty([12*471,9*18],dtype=float)
y = np.empty([12*471,1],dtype=float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day==19 and hour>14:
                continue
            x[month*12+day*20+hour,:] = month_data[month][:,(day*20+hour):(day*20+hour+9)].reshape(1,-1)
            y[month*12+day*20+hour,0] = month_data[month][9,day*20+hour+9]

In [80]:
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [47]:
data

,日期,測站,測項,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,12,15,17,20,22,22,22,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,0.59,0.52,0.41,0.4,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,0.2,0.18,0.12,0.11,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,4.2,2.9,3.4,3,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,14,11,14,12,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,18,14,17,15,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,22,21,29,44,58,50,57,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,2,11,38,56,64,56,57,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,20,19,30,41,44,33,37,36,45,42,49,45,44,41,30,24,13
